In [18]:
# Import libraries
import pandas as pd

# Load the data
file_path = 'Online_Cars_Sale_Marketplace.csv'  # Adjust this if your file path is different
df = pd.read_csv(file_path)

# Step 1: Clean the 'Price' column
# Replace 'Not Priced' with NaN and remove $ and commas
df['Price'] = df['Price'].replace({r'\$': '', ',': '', 'Not Priced': None}, regex=True)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Step 2: Create a 'PopularityScore'
# Popularity Score = ConsumerRating * ConsumerReviews
df['PopularityScore'] = df['ConsumerRating'] * df['ConsumerReviews']

# Step 3: Group by 'Year', 'Make', 'Model'
grouped = df.groupby(['Year', 'Make', 'Model']).agg({
    'PopularityScore': 'sum',
    'ConsumerRating': 'mean',
    'ConsumerReviews': 'sum',
    'Price': 'mean',
    'Mileage': 'mean'
}).reset_index()

# Step 4: Find the most popular car listing
most_popular_car = grouped.sort_values(by='PopularityScore', ascending=False).head(1)

# Display the result
print("Most Popular Car Listing:")
print(most_popular_car)

# Optional: To see Top 5 most popular cars
top5_popular_cars = grouped.sort_values(by='PopularityScore', ascending=False).head(25)
print("Top 5 Most Popular Cars:")
print(top5_popular_cars)


Most Popular Car Listing:
      Year   Make      Model  PopularityScore  ConsumerRating  \
1336  2019  Honda  CR-V EX-L         388800.0             4.8   

      ConsumerReviews         Price       Mileage  
1336            81000  32072.726667  29645.073333  
Top 5 Most Popular Cars:
      Year       Make                   Model  PopularityScore  \
1336  2019      Honda               CR-V EX-L         388800.0   
1335  2019      Honda                 CR-V EX         163296.0   
1144  2018     Toyota                RAV4 XLE         125664.0   
1141  2018     Toyota                 RAV4 LE         121968.0   
1390  2019       Jeep  Grand Cherokee Limited         116121.6   
1384  2019       Jeep        Cherokee Limited         109737.6   
1350  2019      Honda              Pilot EX-L          64224.0   
1023  2018       Jeep  Grand Cherokee Limited          58776.0   
1467  2019      Mazda      CX-5 Grand Touring          57408.0   
969   2018       Ford            F-150 Lariat         

In [39]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9378 entries, 1 to 9378
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   9378 non-null   int64  
 1   Make                   9378 non-null   object 
 2   Model                  9378 non-null   object 
 3   Used/New               9378 non-null   object 
 4   Price                  9373 non-null   float64
 5   ConsumerRating         9378 non-null   float64
 6   ConsumerReviews        9378 non-null   int64  
 7   SellerType             9378 non-null   object 
 8   SellerName             9378 non-null   object 
 9   SellerRating           9378 non-null   float64
 10  SellerReviews          9378 non-null   int64  
 11  StreetName             9378 non-null   object 
 12  State                  9378 non-null   object 
 13  Zipcode                9378 non-null   object 
 14  DealType               9156 non-null   object 
 15  Comf

In [27]:
# Step 1: Select only the important columns
check_df = df[['Year', 'Make', 'Model', 'ConsumerRating', 'ConsumerReviews']].copy()

# Step 2: Group by Year, Make, Model
duplicates = check_df.groupby(['Year', 'Make', 'Model']).nunique()

# Step 3: Find models where either rating or review count varies
problematic_models = duplicates[(duplicates['ConsumerRating'] > 1) | (duplicates['ConsumerReviews'] > 1)].reset_index()

# Step 4: Show the problematic models
print(f"Number of problematic models: {problematic_models.shape[0]}")
print(problematic_models)

# Step 5: (Optional) Show the actual raw listings for these models
# Create a filter to select those problematic models from the original data
# ----> Use only ['Year', 'Make', 'Model'] when merging to avoid bringing in extra columns
problematic_rows = df.merge(problematic_models[['Year', 'Make', 'Model']], on=['Year', 'Make', 'Model'], how='inner')

# Step 6: Display the raw rows
print("\nProblematic listings:")
print(problematic_rows[['Year', 'Make', 'Model', 'ConsumerRating', 'ConsumerReviews']].sort_values(by=['Year', 'Make', 'Model']))


Number of problematic models: 1
   Year       Make        Model  ConsumerRating  ConsumerReviews
0  2015  Chevrolet  Equinox 1LT               1                2

Problematic listings:
   Year       Make        Model  ConsumerRating  ConsumerReviews
0  2015  Chevrolet  Equinox 1LT             4.6              295
1  2015  Chevrolet  Equinox 1LT             4.6              296


In [31]:

# Find the index of the row with lower ConsumerReviews
drop_idx = problematic_rows.sort_values('ConsumerReviews').index[0]

# Drop it from the main dataframe
df = df.drop(index=drop_idx)

In [37]:
# Group again by Year, Make, Model
grouped = df.groupby(['Year', 'Make', 'Model']).agg({
    'PopularityScore': 'sum',
    'ConsumerRating': 'mean',
    'ConsumerReviews': 'sum',
    'Price': 'mean',
    'Mileage': 'mean'
}).reset_index()

# Sort by Popularity Score descending
most_popular_car = grouped.sort_values(by='PopularityScore', ascending=False).head(25)

# Display the most popular car
print("Most Popular Car Listing After Cleaning:")
print(most_popular_car)


Most Popular Car Listing After Cleaning:
      Year       Make                   Model  PopularityScore  \
1336  2019      Honda               CR-V EX-L         388800.0   
1335  2019      Honda                 CR-V EX         163296.0   
1144  2018     Toyota                RAV4 XLE         125664.0   
1141  2018     Toyota                 RAV4 LE         121968.0   
1390  2019       Jeep  Grand Cherokee Limited         116121.6   
1384  2019       Jeep        Cherokee Limited         109737.6   
1350  2019      Honda              Pilot EX-L          64224.0   
1023  2018       Jeep  Grand Cherokee Limited          58776.0   
1467  2019      Mazda      CX-5 Grand Touring          57408.0   
969   2018       Ford            F-150 Lariat          54902.4   
1142  2018     Toyota            RAV4 Limited          48048.0   
1540  2019        RAM            1500 Laramie          44556.0   
1252  2019   Cadillac              XT5 Luxury          44537.2   
1593  2019     Toyota              